# Number theory and a Google recruitment puzzle

## Find the first 10-digit prime in the decimal expansion of 17π

Description: The first 5 digits in the decimal expansion of π are 14159. The first 4-digit prime in the decimal expansion of π are 4159. You are asked to find the first 10-digit prime in the decimal expansion of 17π. 

There are three main steps to this question.   
    1) Generate an arbitrary large expansion of a mathematical expression  
    2) Check if a number is prime  
    3) Generate sliding windows of a specified width from a long iterable  
  
We will start with the first step. The goal is to return an arbitrary expansion (expansion after the decimal) of a mathematical expression. The user inputs will be the mathematical expression along with the multiplier (can be 1) and the number of digits of the expansion.
  
I first tried using the decimal library to expand the mathematical expression "pi", but I realized I was approximating pi by using the expression "355/113" and I could not find a more accurate way to expand pi, so I opted to use the sympy library. I used the decimal library to expand "e". 

In [173]:
from decimal import *
from sympy import *

def expansion(multiplier, expression, n):
    """
    Input: mathematical expression, a multiplier of the expression, and number of digits of prime number
    Behavior: expands mathematical expression
    Output: an arbitrary expansion (specified by user) of a mathematical expressions pi or e 
    """
    if expression == "pi":
        # N function captures number of digits, so use log formula to capture number of specified digits after decimal
        x = N(multiplier*pi, n + log(multiplier*pi, 10) + 1)
        # obtain digits after decimal and return as integer (integers that start with 0 will not capture 0)
        before, after = str(x).split('.')
        return int(after)
    if expression == "e":
        # set number of digits for expansion
        getcontext().prec = n
        x = (Decimal(1).exp())*multiplier
        before, after = str(x).split('.')
        return int(after)
    # can only generate expansions of pi and e
    else:
        return "Can not solve"

The next step is to determine if a number is prime. Using mathematical intuition, we can loop through divisors in the range from 2 to the square root of the number rounded up using the ceiling function from the math module. For example, let's take the number 11. The square root of 11 rounded up is 4, so the loop will run from 2 to 4. The mathematical intution behind this is that a number that is not prime will have atleast one factor that is less than its square root (and one factor greater). So, if a number has no factors less than its square root, than it must be prime. 

In [215]:
from math import sqrt, ceil

def prime_num(n):
    """
    Input: number
    Behavior: returns true if number is prime
    Output: true or false
    """
    # prime numbers have to be greater than 1
    if n > 1:
        
        # 
        for number in range(2, ceil(sqrt(n))):
            # return to outer for loop and increment by one if input number is not divisible by number
            if (n % number) == 0:
                # number is not prime (divisible by a number other than one and itself)
                return False
        else:
            # number is prime, so return true
            return True
    else: 
        # return False if number is not greater than 1
        return False
print(prime_num(3))

True


I tried a lot of different things to generate sliding windows including using itertools, but decided that using list comprehensions would be equally succinct in accomplishing this task. 

In [218]:
def sliding_window(iterable, size):
    """
    Input: integer and size of window 
    Behavior: generates sliding windows of a specified width from a long iterable
    Output: list of windows
    """
    window_list = []
    # length of number has to be greater than the window size
    if len(str(iterable)) >= size:
        # make number into list of its digits
        it_list = [str(x) for x in str(iterable)]
        
        # create window by grabbing element in range of current element through the elements in the size of the window
        # create separate window as a list for each element in the list 
        windows = [it_list[x:x+size] for x in range(len(it_list) - size + 1)]
        
        # convert list of integers in window to a single integer and append to list
        for window in windows:
            a_string = "".join(window)
            an_integer = int(a_string)
            window_list.append(an_integer)
            
        # returns list of integers
        return window_list
    
    else:
        return "Size of window bigger than iterable"

In [219]:
def size_digit_prime(size, multiplier, expression, digits):
    """
    Input: size of window, multiplier of expression, mathematical expression, and number of digits to expand
    Behavior: returns a specified digit-length prime in the decimal expansion of a mathematical expression like pi
    Output: number
    """
    expanded = expansion(multiplier, expression, digits)
    expansion_window = sliding_window(expanded, size)
    for number in expansion_window:
        # ignores numbers that start with 0
        if len(str(number)) == 10:
            if prime_num(number):
                return number
    
print(size_digit_prime(10, 17, "pi", 110))

8649375157


In [170]:
import unittest

class TestNotebook(unittest.TestCase):
    
    def test_expansion(self):
        """test expansion."""
        self.assertEqual(expansion(1, "pi", 5), 14159)
        self.assertEqual(expansion(17, "pi", 9), 407075111)
        
    def test_prime_num(self):
        """test prime_num."""
        self.assertEqual(prime_num(1), False)
        self.assertEqual(prime_num(4159), True)
        
    def test_sliding_window(self):
        """test sliding_window."""
        self.assertEqual(sliding_window(407075111, 4), [4070, 707, 7075, 751, 7511, 5111])
        self.assertEqual(sliding_window(1234, 5), "Size of window bigger than iterable")

unittest.main(argv=[''], verbosity=2, exit=False)

test_expansion (__main__.TestNotebook)
test expansion. ... ok
test_prime_num (__main__.TestNotebook)
test prime_num. ... ok
test_sliding_window (__main__.TestNotebook)
test sliding_window. ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.008s

OK
